In [102]:
import os
from glob import glob
from chord_extractor.extractors import Chordino
import music21

In [21]:
folder = "./data/Complete Examples Melodies"

files = [y for x in os.walk(folder) for y in glob(os.path.join(x[0], '*.mid'))]

In [22]:
files

['./data/Complete Examples Melodies/Doug McKenzie/I Thought About You.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Alice In Wonderland.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Autumn Leaves.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Blue Room.mid',
 './data/Complete Examples Melodies/Doug McKenzie/But Beautiful.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Come Rain Or Come Shine (1).mid',
 './data/Complete Examples Melodies/Doug McKenzie/Come Rain Or Come Shine (2).mid',
 './data/Complete Examples Melodies/Doug McKenzie/Come Sunday.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Cry Me A River.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Dancing On The Ceiling.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Estate.mid',
 './data/Complete Examples Melodies/Doug McKenzie/Have You Met Miss Jones (1).mid',
 './data/Complete Examples Melodies/Doug McKenzie/Have You Met Miss Jones (2).mid',
 './data/Complete Example

In [27]:
orig_files = [(f.split('/')[-1], f.split('/')[-2]) for f in files if 'Real Book' in f]
len(orig_files)

207

In [28]:
orig_files

[('A Felicidade.mid', 'Real Book'),
 ('A Foggy Day.mid', 'Real Book'),
 ('A Night In Tunisia.mid', 'Real Book'),
 ('Accentuate The Positive.mid', 'Real Book'),
 ('Afro Blue.mid', 'Real Book'),
 ("After You've Gone.mid", 'Real Book'),
 ("Ain't Misbehavin'.mid", 'Real Book'),
 ('Airegin.mid', 'Real Book'),
 ('Alice In Wonderland.mid', 'Real Book'),
 ('All Blues.mid', 'Real Book'),
 ('All Of Me.mid', 'Real Book'),
 ('All Or Nothing At All.mid', 'Real Book'),
 ('All The Things You Are.mid', 'Real Book'),
 ('Alone Together.mid', 'Real Book'),
 ('Another You.mid', 'Real Book'),
 ('Anthropology.mid', 'Real Book'),
 ('April In Paris.mid', 'Real Book'),
 ('Autumn Leaves.mid', 'Real Book'),
 ("Bags' Groove.mid", 'Real Book'),
 ('Beautiful Friendship.mid', 'Real Book'),
 ('Beautiful Love.mid', 'Real Book'),
 ('Bemsha Swing.mid', 'Real Book'),
 ("Bessie's Blues.mid", 'Real Book'),
 ('Bewitched.mid', 'Real Book'),
 ('Black Narcissus.mid', 'Real Book'),
 ('Black Orpheus.mid', 'Real Book'),
 ('Blue I

In [29]:
impro_files = [(f.split('/')[-1], f.split('/')[-2]) for f in files if 'Real Book' not in f]
len(impro_files)

249

In [33]:
impro_files.sort()
impro_files

[('A Felicidade.mid', 'JazzPage'),
 ("Ain't Misbehavin'.mid", 'MidKar'),
 ('Alice In Wonderland.mid', 'Doug McKenzie'),
 ('All The Things You Are (1).mid', 'JazzPage'),
 ('All The Things You Are.mid', 'MidKar'),
 ('Alone Together (2).mid', 'JazzPage'),
 ('Alone Together (2).mid', 'Oocities'),
 ('April In Paris.mid', 'Jazz-Midi'),
 ('Art Pepper - Anthropology.mid', 'Weimar DB'),
 ('Art Pepper - In A Mellow Tone.mid', 'Weimar DB'),
 ('Autumn Leaves (1).mid', 'Jazz-Midi'),
 ('Autumn Leaves (2).mid', 'Jazz-Midi'),
 ('Autumn Leaves.mid', 'Doug McKenzie'),
 ('Bemsha Swing.mid', 'MidKar'),
 ('Ben Webster - Bye Bye Blackbird.mid', 'Weimar DB'),
 ('Ben Webster - Night And Day.mid', 'Weimar DB'),
 ('Benny Carter - Just Friends.mid', 'Weimar DB'),
 ('Benny Carter - Long Ago And Far Away (1).mid', 'Weimar DB'),
 ('Benny Carter - Long Ago And Far Away (2).mid', 'Weimar DB'),
 ('Bix Beiderbecke - Margie.mid', 'Weimar DB'),
 ('Black Narcissus.mid', 'JazzPage'),
 ('Black Narcissus.mid', 'MidKar'),
 ('

In [70]:
class ScoreParams:
	'''
	Define scores for each parameter
	'''
	def __init__(self,gap,match,mismatch):
		self.gap = gap
		self.match = match
		self.mismatch = mismatch

	def misMatchChar(self,x,y):
		if x != y:
			return self.mismatch
		else:
			return self.match

def getMatrix(sizeX,sizeY,gap):
	'''
	Create an initial matrix of zeros, such that its len(x) x len(y)
	'''
	matrix = []
	for i in range(len(sizeX)+1):
		subMatrix = []
		for j in range(len(sizeY)+1):
			subMatrix.append(0)
		matrix.append(subMatrix)

	# Initializing the first row and first column with the gap values
	for j in range(1,len(sizeY)+1):
		matrix[0][j] = j*gap
	for i in range(1,len(sizeX)+1):
		matrix[i][0] = i*gap
	return matrix

def getTraceBackMatrix(sizeX,sizeY):
	'''
	Create an initial matrix of zeros, such that its len(x) x len(y)
	'''
	matrix = []
	for i in range(len(sizeX)+1):
		subMatrix = []
		for j in range(len(sizeY)+1):
			subMatrix.append('0')
		matrix.append(subMatrix)

	# Initializing the first row and first column with the up or left values
	for j in range(1,len(sizeY)+1):
		matrix[0][j] = 'left'
	for i in range(1,len(sizeX)+1):
		matrix[i][0] = 'up'
	matrix[0][0] = 'done'
	return matrix


def globalAlign(x,y,score):
	'''
	Fill in the matrix with alignment scores
	'''
	matrix = getMatrix(x,y,score.gap)
	traceBack = getTraceBackMatrix(x,y)

	for i in range(1,len(x)+1):
		for j in range(1,len(y)+1):
			left = matrix[i][j-1] + score.gap
			up = matrix[i-1][j] + score.gap
			diag = matrix[i-1][j-1] + score.misMatchChar(x[i-1],y[j-1])
			matrix[i][j] = max(left,up,diag)
			if matrix[i][j] == left:
				traceBack[i][j] = 'left'
			elif matrix[i][j] == up:
				traceBack[i][j] = 'up'
			else:
				traceBack[i][j] = 'diag'
	return matrix,traceBack

def getAlignedSequences(x,y,matrix,traceBack):
	'''
	Obtain x and y globally aligned sequence arrays using the bottom-up approach
	'''
	xSeq = []
	ySeq = []
	i = len(x)
	j = len(y)
	while(i > 0 or j > 0):
		if traceBack[i][j] == 'diag':
			# Diag is scored when x[i-1] == y[j-1]
			xSeq.append(x[i-1])
			ySeq.append(y[j-1])
			i = i-1
			j = j-1
		elif traceBack[i][j] == 'left':
			# Left holds true when '-' is added from x string and y[j-1] from y string
			xSeq.append('-')
			ySeq.append(y[j-1])
			j = j-1
		elif traceBack[i][j] == 'up':
			# Up holds true when '-' is added from y string and x[j-1] from x string
			xSeq.append(x[i-1])
			ySeq.append('-')
			i = i-1
		elif traceBack[i][j] == 'done':
			# Break condition when we reach the [0,0] cell of traceback matrix
			break
	return xSeq,ySeq

def printMatrix(matrix):
	'''
	Create a custom function to print the matrix
	'''
	for i in range(len(matrix)):
		print(matrix[i])
	print()

'''
Driver Code:
'''
x = 'casa'
y = 'aacasa'
print('Input sequences are: ')
print(x)
print(y)
print()
score = ScoreParams(-2,1,-1)
matrix,traceBack = globalAlign(x,y,score)

print('Printing the score matrix:')
printMatrix(matrix)

print('Printing the trace back matrix:')
printMatrix(traceBack)

xSeq,ySeq = getAlignedSequences(x,y,matrix,traceBack)

print('The globally aligned sequences are:')
print(*xSeq[::-1])
print(*ySeq[::-1])

Input sequences are: 
casa
aacasa

Printing the score matrix:
[0, -2, -4, -6, -8, -10, -12]
[-2, -1, -3, -3, -5, -7, -9]
[-4, -1, 0, -2, -2, -4, -6]
[-6, -3, -2, -1, -3, -1, -3]
[-8, -5, -2, -3, 0, -2, 0]

Printing the trace back matrix:
['done', 'left', 'left', 'left', 'left', 'left', 'left']
['up', 'diag', 'left', 'diag', 'left', 'left', 'left']
['up', 'diag', 'diag', 'left', 'diag', 'left', 'left']
['up', 'up', 'up', 'diag', 'left', 'diag', 'left']
['up', 'up', 'diag', 'up', 'diag', 'left', 'diag']

The globally aligned sequences are:
- - c a s a
a a c a s a


In [104]:
chordino = Chordino(roll_on=1)

path = './data/Complete Examples'

score = ScoreParams(-2, 1, -1)

for orig_filename in orig_files[:1]:
    
    print(orig_filename[0])
    print('-----')
    
    filepath = os.path.join(path, orig_filename[1], orig_filename[0])
    
    score = music21.converter.parse(filepath)
    key = score.analyze('key')
    print(key.tonic.name, key.mode)
    
    orig_conversion_file_path = chordino.preprocess(filepath)
    orig_chords = chordino.extract(orig_conversion_file_path)
    orig_chord_names = [c.chord for c in orig_chords]
    
    print(orig_conversion_file_path)
    print("-".join(orig_chord_names))
    print('---------------')
    
    chord_extractor.clear_conversion_cache()
        
    improv_filenames = [os.path.join(path, g[1], g[0]) for g in impro_files if g[0] == f[0]]
    
    for impro_filepath in improv_filenames:
        
        score = music21.converter.parse(impro_filepath)
        key = score.analyze('key')
        print(key.tonic.name, key.mode)
        
        print(impro_filepath)
        print('-----')
        
        impro_conversion_file_path = chordino.preprocess(impro_filepath)
        impro_chords = chordino.extract(impro_conversion_file_path)
        impro_chord_names = [c.chord for c in impro_chords]
        
        print(impro_conversion_file_path)
        print("-".join(impro_chord_names))
        print('---------------')
    
        matrix, traceBack = globalAlign(orig_chord_names, impro_chord_names, score)

        xSeq, ySeq = getAlignedSequences(orig_chord_names, impro_chord_names, matrix, traceBack)

        print('The globally aligned sequences are:')
        print(*xSeq[::-1])
        print('-----')
        print(*ySeq[::-1])
        print('--------------------------------')
        
        chord_extractor.clear_conversion_cache()

A Felicidade.mid
-----


/media/manu/Data/PycharmProjects/thesis/venv/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=2, channel=None, data=b'Bater\xeda'>; getting generic Instrument
  warnings.warn(
/media/manu/Data/PycharmProjects/thesis/venv/lib/python3.8/site-packages/music21/midi/translate.py:790: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent PROGRAM_CHANGE, track=2, channel=10, data=0>; getting generic UnpitchedPercussion
  warnings.warn(
/media/manu/Data/PycharmProjects/thesis/venv/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Melod\xeda'>; getting generic Instrument
  warnings.warn(


E- major
/tmp/extractor/A Felicidade
N-D-Cm7-Gm7-D7-Gm7-Ebmaj7-Fm7-Dm7b5-Cm7-Fm7b5-C#-A7-Abmaj7-G-Cm7-Dm7b5-G7-Cm7-Dm7b5-Eb6-Bbm7-Eb7-Fm7-Bb7-Eb6-Am7b5-Daug-Am7b5-Daug-Gm7b5-C7-Fm7b5-Eb6-Dm7b5-G7-Cm7-Dm7b5-G7-Cm7-Gm7-D7-Gm7-Gm-Eb6-C7-Fm7-Dm7b5-Cm7-B7-C#-Eb7-Abmaj7-G7-Cm7-Gm7-Cm7-Dm7b5-Cm7-Dm7b5-Cm7-Dm7b5-Cm7-Gm7-D7-Gm7-Gm-Eb6-C7-Fm7-Dm7b5-Cm7-F7-Bbm7-A7-Abmaj7-G7-Cm7-Dm7b5-G7-Cm7-Dm7b5-Cm7-Ab7-Bb6-Bbm7-Eb7-Fm7-Bb7-Eb6-Am7b5-Cm7b5/Bb-Gm7b5-C7-Fm7b5-Eb6-Dm7b5-G7-Cm7-Dm7b5-Cm7-Gm7-D7-Gm7-Ebmaj7-C7-Fm-Cm7-Bm7b5-Bbm7-A7-Cm-G7-Cm7-Dm7b5-G7-Cm7-Dm7b5-Cm7-Dm7b5-Cm7-Dm7b5-Cm7-N
---------------
A minor
./data/Complete Examples/JazzPage/A Felicidade.mid
-----
/tmp/extractor/A Felicidade
N-Am-Em-B7-Em-C-Bm7b5-Dm7b5-Am7-Abm7-Gm7-C7-F-E7-Am-D7-Am7-Bm7b5-Am-Dm-G7-Cmaj7-Gm7-C7-F6-G7-Ebaug-C6-B7-Em7-G7-C-Am7-Dm7-Am7-Bm7b5-Am-Em-B7-Em-Dm7-Fm7b5-C-Bm7b5-E7-Am7-Abm7-Gm7-C7-F-E7-Am-D7-Am7-Bm7b5-E7-Am-Dm-G7-G6-Gm7-Bbm7b5-F-G7-Gaug-C6-F#m7-Baug-Em7-Bbm6-Dm7-C-Am7-Dm7-E7-Am7-Bm7b5-Am-Em-B7-Em-Dm7-Fm7b5-C-Bm7b

AttributeError: 'Score' object has no attribute 'gap'

In [79]:
import pychord

In [82]:
c = pychord.ChordProgression(['C', 'Am', 'F', 'G'])

In [84]:
c.transpose(1)

In [85]:
c

<ChordProgression: Db | Bbm | Gb | Ab>